<a href="https://colab.research.google.com/github/aniketsharma00411/sign-language-to-text-translator/blob/main/transfer_learning_evaluation_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [ ]:
from google.colab import files
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import applications

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import time

In [ ]:
if not os.path.exists(os.path.expanduser('~')+'/.kaggle'):
    ! mkdir ~/.kaggle
os.chdir(os.path.expanduser('~')+'/.kaggle')
if not os.path.exists(os.path.expanduser('~')+'/.kaggle/kaggle.json'):
    kaggle_api_file = files.upload()
    ! kaggle datasets download -d grassknoted/asl-alphabet
    ! unzip -q asl-alphabet.zip
    ! rm -rf asl_alphabet_train/asl_alphabet_train/del

Saving kaggle.json to kaggle.json
 98% 1.01G/1.03G [00:05<00:00, 201MB/s]
100% 1.03G/1.03G [00:05<00:00, 196MB/s]


In [ ]:
! ls

asl_alphabet_test  asl_alphabet_train  asl-alphabet.zip  kaggle.json


# Creating Models

In [ ]:
def get_data(preprocessing_function):
    image_gen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=(0.2, 1),
                                   shear_range=45,
                                   zoom_range=(0.5, 1.5),
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   preprocessing_function=preprocessing_function,
                                   validation_split=0.2)

    train_data = 'asl_alphabet_train/asl_alphabet_train'

    train_gen = image_gen.flow_from_directory(train_data,
                                              target_size=(224, 224),
                                              class_mode='categorical',
                                              color_mode='rgb',
                                              shuffle=True,
                                              batch_size=128,
                                              seed=0,
                                              subset='training')

    val_gen = image_gen.flow_from_directory(train_data,
                                            target_size=(224, 224),
                                            class_mode='categorical',
                                            color_mode='rgb',
                                            shuffle=True,
                                            batch_size=128,
                                            seed=0,
                                            subset='validation')

    return train_gen, val_gen

In [ ]:
def get_model(model):
    kwargs =    {'include_top':False,
                 'weights':'imagenet',
                 'input_shape':(224, 224, 3),
                 'pooling':'avg'}
    
    base_model = model(**kwargs)
    
    end_model = models.Sequential()
    end_model.add(layers.Flatten(input_shape=base_model.output_shape[1:]))
    end_model.add(layers.Dense(64))
    end_model.add(layers.LeakyReLU())
    end_model.add(layers.Dense(64))
    end_model.add(layers.LeakyReLU())
    end_model.add(layers.Dense(28, activation='softmax'))

    model = models.Model(inputs=base_model.input, outputs=end_model(base_model.output))

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
models_dict = {
    'Xception': {'model': applications.xception.Xception,
                 'preprocess_func': applications.xception.preprocess_input},
    'VGG16': {'model': applications.vgg16.VGG16,
              'preprocess_func': applications.vgg16.preprocess_input},
    'VGG19': {'model': applications.vgg19.VGG19,
              'preprocess_func': applications.vgg19.preprocess_input},
    'ResNet50': {'model': applications.resnet.ResNet50,
                 'preprocess_func': applications.resnet.preprocess_input},
    'ResNet101': {'model': applications.resnet.ResNet101,
                  'preprocess_func': applications.resnet.preprocess_input},
    'ResNet152': {'model': applications.resnet.ResNet152,
                  'preprocess_func': applications.resnet.preprocess_input},
    'ResNet50V2': {'model': applications.resnet_v2.ResNet50V2,
                   'preprocess_func': applications.resnet_v2.preprocess_input},
    'ResNet101V2': {'model': applications.resnet_v2.ResNet101V2,
                    'preprocess_func': applications.resnet_v2.preprocess_input},
    'ResNet152V2': {'model': applications.resnet_v2.ResNet152V2,
                    'preprocess_func': applications.resnet_v2.preprocess_input},
    'InceptionV3': {'model': applications.inception_v3.InceptionV3,
                    'preprocess_func': applications.inception_v3.preprocess_input},
    'InceptionResNetV2': {'model': applications.inception_resnet_v2.InceptionResNetV2,
                          'preprocess_func': applications.inception_resnet_v2.preprocess_input},
    'MobileNet': {'model': applications.mobilenet.MobileNet,
                  'preprocess_func': applications.mobilenet.preprocess_input},
    'MobileNetV2': {'model': applications.mobilenet_v2.MobileNetV2,
                    'preprocess_func': applications.mobilenet_v2.preprocess_input},
    'DenseNet121': {'model': applications.densenet.DenseNet121,
                    'preprocess_func': applications.densenet.preprocess_input},
    'DenseNet169': {'model': applications.densenet.DenseNet169,
                    'preprocess_func': applications.densenet.preprocess_input},
    'DenseNet201': {'model': applications.densenet.DenseNet201,
                    'preprocess_func': applications.densenet.preprocess_input},
    'NASNetMobile': {'model': applications.nasnet.NASNetMobile,
                     'preprocess_func': applications.nasnet.preprocess_input},
    'EfficientNetB0': {'model': applications.efficientnet.EfficientNetB0,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB1': {'model': applications.efficientnet.EfficientNetB1,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB2': {'model': applications.efficientnet.EfficientNetB2,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB3': {'model': applications.efficientnet.EfficientNetB3,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB4': {'model': applications.efficientnet.EfficientNetB4,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB5': {'model': applications.efficientnet.EfficientNetB5,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB6': {'model': applications.efficientnet.EfficientNetB6,
                       'preprocess_func': applications.efficientnet.preprocess_input},
    'EfficientNetB7': {'model': applications.efficientnet.EfficientNetB7,
                       'preprocess_func': applications.efficientnet.preprocess_input}
}

# Training Models

In [ ]:
for name, model in models_dict.items():
    print(name)

    train_gen, val_gen = get_data(model['preprocess_func'])
    m = get_model(model['model'])

    start = time.perf_counter()

    history = m.fit(train_gen,
                    epochs=1,
                    validation_data=val_gen)

    duration = time.perf_counter() - start

    models_dict[name]['time'] = duration
    models_dict[name]['val_acc'] = history.history['val_accuracy'][-1]
    
    print()

Xception
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1056s 2s/step - loss: 1.6490 - accuracy: 0.5095 - val_loss: 1.6198 - val_accuracy: 0.5070

VGG16
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1079s 2s/step - loss: 1.7234 - accuracy: 0.4907 - val_loss: 1.5898 - val_accuracy: 0.5186

VGG19
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1084s 2s/step - loss: 1.7223 - accuracy: 0.4912 - val_loss: 1.6270 - val_accuracy: 0.5135

ResNet50
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1032s 2s/step - loss: 1.3508 - accuracy: 0.5900 - val_loss: 1.2841 - val_accuracy: 0.5951

ResNet101
Found 67200 images belonging to 28 classes.
Found 16800 images belonging

# Evaluating and Visualizing results

In [ ]:
model_results = []

for name, _ in models_dict.items():
    model_results.append([name,
                          models_dict[name]['val_acc'],
                          models_dict[name]['time']])
    
results = pd.DataFrame(model_results,
                       columns = ['Model', 'Validation Accuracy', 'Training Time (sec.)'])
results = results.sort_values(by='Validation Accuracy', ascending=False).reset_index(drop=True)

results

,Model,Validation Accuracy,Training Time (sec.)
0,MobileNet,0.659464,933.186254
1,ResNet152,0.626190,1177.023145
2,EfficientNetB5,0.625714,1082.315889
3,EfficientNetB7,0.616905,1251.437560
4,DenseNet201,0.614940,1067.191720
5,ResNet101,0.610059,1099.715862
6,ResNet101V2,0.604643,1056.137887
7,EfficientNetB1,0.601786,969.576498
8,ResNet50,0.595119,1033.249712
9,DenseNet169,0.594345,1027.773288
